# NSIDC DAAC Customize and Access Data Tutorial
#### This tutorial will walk you though how to access NSIDC DAAC data using spatial and temporal filters, as well as how to request customization services including subsetting, reformatting, and reprojection. 

## Import packages


In [1]:
import requests, getpass, json, zipfile, io
from xml.etree import ElementTree

## Create a token

We will generate a token needed in order to access data using your Earthdata Login credentials, and we will apply that token to the following queries.

In [2]:
# Earthdata Login credentials

uid = input('Earthdata Login user name: ')
pswd = getpass.getpass('Earthdata Login password: ')


Earthdata Login user name: amy.steiker
Earthdata Login password: ········


In [3]:
# Request token from Common Metadata Repository using Earthdata credentials
token_url = 'https://api.echo.nasa.gov/echo-rest/tokens'
hostname = socket.gethostname()
ip = socket.gethostbyname(hostname)

data = {
    'token': {
        'username': username,
        'password': password,
        'client_id': 'NSIDC_client_id',
        'user_ip_address': ip
    }
}
headers={'Accept': 'application/json'}
response = requests.post(token_api_url, json=data, headers=headers)
token = json.loads(response.content)['token']['id']

AC5DFB22-89B3-808E-BB0F-18960D364EFB


## Select a data set of interest and determine the number and size of granules available within a time range and location.

#### Let's begin discovering an NSIDC DAAC data set by first inputting the data set of interest and determining the most recent version number. We will also find out how many data granules exist over an area and time of interest. The Common Metadata Repository is queried to explore this information.  

In [138]:
# Input data set short name (e.g. ATL03) of interest here.

short_name = input('Input short name, e.g. ATL03, here: ')

Input short name, e.g. ATL03, here: MOD10A1


In [139]:
# For restricted collections
# First determine the latest verion number by querying CMR collection metadata.


# mr_url = 'https://cmr.earthdata.nasa.gov/search/collections.json?short_name=' + short_name + '&token=' + token
# cmeta = requests.get(cmr_url)
 
# with urllib.request.urlopen(cmr_url) as url:
#    cmeta_json = json.loads(url.read().decode())

In [140]:
# Get json response from CMR collection metadata

params = {
    'short_name': short_name
}

cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
response = requests.get(cmr_collections_url, params=params)
results = json.loads(response.content)

# Find all instances of 'version_id' in metadata and print most recent version number

versions = [el['version_id'] for el in results['feed']['entry']]
latest_version = max(versions)
print('The most recent version of ', short_name, ' is ', latest_version)

The most recent version of  MOD10A1  is  6


#### Now that we have the most recent version of this data set, let's determine the number of granules available over our area and time of interest.

In [133]:
#https://cmr.earthdata.nasa.gov/search/granules?

#Input bounding box

LL_lon = input('Input lower left longitude in decimal degrees: ')
LL_lat = input('Input lower left latitude in decimal degrees: ')
UR_lon = input('Input upper right longitude in decimal degrees: ')
UR_lat = input('Input upper right latitude in decimal degrees: ')

bounding_box = LL_lon + ',' + LL_lat + ',' + UR_lon + ',' + UR_lat
           
#Maybe a way to upload a shapefile and output the coordinates?

Input lower left longitude in decimal degrees: -120.234234
Input lower left latitude in decimal degrees: 40.23423
Input upper right longitude in decimal degrees: -115.34234
Input upper right latitude in decimal degrees: 42.23423


In [142]:
#Input temporal range 

start_date = input('Input start date in yyyy-MM-dd format: ')
start_time = input('Input start time in HH:mm:ss format: ')
end_date = input('Input end date in yyyy-MM-dd format: ')
end_time = input('Input end time in HH:mm:ss format: ')

temporal = start_date + 'T' + start_time + 'Z' + ',' + end_date + 'T' + end_time + 'Z'


Input start date in yyyy-MM-dd format: 2019-03-01
Input start time in HH:mm:ss format: 00:00:00
Input end date in yyyy-MM-dd format: 2019-03-20
Input end time in HH:mm:ss format: 23:59:59


In [143]:
# Query number of granules (paging over results)


granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'
params = {
    'short_name': short_name,
    'version': latest_version,
    'bounding_box': bounding_box,
    'temporal': temporal,
    'page_size': 100,
    'page_num': 1
}
granules = []
while True:
    response = requests.get(granule_search_url, params=params, headers=headers)
    results = json.loads(response.content)

    if len(results['feed']['entry']) == 0:
        # Out of results, so break out of loop
        break

    # Collect results and increment page_num
    granules.extend(results['feed']['entry'])
    params['page_num'] += 1

print('There are', len(granules), 'granules of', short_name, 'version', latest_version, 'over my area and time of interest.')


There are 40 granules of MOD10A1 version 6 over my area and time of interest.


#### We will now query the average size of those granules as well as the total volume. 

In [148]:
from statistics import mean

granule_sizes = [float(granule['granule_size']) for granule in granules]

print(f'The average size of each granule is {mean(granule_sizes):.2f} MB and the total size of all {len(granules)} granules is {sum(granule_sizes):.2f} MB')


The average size of each granule is 4.546394425 MB and the total size of all  40 granules is 181.85577700000002 MB


#### The NSIDC DAAC supports customization services on many of our NASA Earthdata mission collections. Let's discover if our data set has these services available. 

In [203]:
# Query service capability URL 
capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{short_name}.{latest_version}.xml'

response = requests.get(capability_url)

root = ElementTree.fromstring(response.content)

subset_agents = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]
    
if len(subset_agents) < 1 :
    print('No services exist for', short_name, 'version', latest_version)
else:
    spatial_subsetting = any(subset_agent['spatialSubsetting'] == 'true'
            for subset_agent in subset_agents)
    temporal_subsetting = any(subset_agent['temporalSubsetting'] == 'true'
            for subset_agent in subset_agents)
    print(f"Spatial subsetting: {'Yes' if spatial_subsetting else 'No'}")
    print(f"Temporal subsetting: {'Yes' if temporal_subsetting else 'No'}")

{'id': 'HEG', 'spatialSubsetting': 'true', 'temporalSubsetting': 'false', 'type': 'both', 'maxGransSyncRequest': '100', 'maxGransAsyncRequest': '2000'}
True
False


In [ ]:

#for SubsetAgent in root.findall('SubsetAgent'):
#    print(id)

#'SubsetAgent' in child.tagroot.iter('*')

#if not child in root.iter('SubAgent'):
#    print('No customization services exist for', short_name, 'version', latest_version)
#else: 
#    for child in root.iter('SubAgent'):
#        print(child.attrib['*'])


#for child in root.iter('SubAgent'):
#    print(child.attrib['*'])

For data sets with the services we just queried, let's explore the specific service options available for this data set and select which of these services we want to request.

In [ ]:
response = requests.get(capability_url)
root = ElementTree.fromstring(response.content)
for child in root.iter('Format'):
    print(child.attrib['value'])

    
#User input: Do you want reformatting? If so, select __ 

## Choose request method

There are two main access methods that the NSIDC DAAC supports through our Application Programming Interface (API).

The first is synchronous: The data request is processed on the fly. Upon completion, data are downloaded directly to this directory as a single zip file. 

The second method is asynchronous: The data request is processed at NSIDC and sent to you via email. The email contains zip file(s) and a link to the order information, along with individual output download links.

In [ ]:
# Before we download, we need to determine the max number of sync granules

# 


# Download data test (still in progress)


# base URL: https://n5eil02u.ecs.nsidc.org/egi/request?
# short_name=GLAH12
# version=034
# bounding_box=-50.33333,68.56667,-49.33333,69.56667
# bbox=-50.33333,68.56667,-49.33333,69.56667
# time=2009-01-01T00:00:00,2009-12-31T23:59:59
# format=TABULAR_ASCII
# token=TOKEN-FROM-STEP-2

access_url = 'https://n5eil02u.ecs.nsidc.org/egi/request?' + 'short_name=GLAH12&' + 'version=034&' + 'bounding_box=-50.33333,68.56667,-49.33333,69.56667&' + 'bbox=-50.33333,68.56667,-49.33333,69.56667&' + 'time=2009-01-01T00:00:00,2009-12-31T23:59:59&' + 'format=TABULAR_ASCII&' + 'token=' + token
print(access_url)

In [ ]:
r = requests.get(access_url, allow_redirects=True)
#open('google.ico', 'wb').write(r.content)
#resp = requests.get("http://www.example.com", 
                    #params = {"name":"Daniel", "id": 123456})
    
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()


#with open("data_output",'wb') as f: 
  
    # Saving received content as a png file in 
    # binary format 
  
    # write the contents of the response (r.content) 
    # to a new file in binary mode. 
    #f.write(r.content) 
  
